In [232]:
import pandas as pd
import numpy as np
import ast
import re
from pathlib import Path
from functools import partial
import requests
import urllib

In [233]:
df1 = pd.read_csv("processed_data_till_60000.csv")
df2 = pd.read_csv("processed_data_60000_120000.csv")
df3 = pd.read_csv("processed_data_120000_150000.csv")
df4 = pd.read_csv("processed_data_150000_190000.csv")
df5 = pd.read_csv("processed_data_190000_260000.csv")

In [234]:
df = pd.concat([ df1 , df2 , df3 ,df4 , df5  ])

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 260000 entries, 0 to 69999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      260000 non-null  int64 
 1   image_link      260000 non-null  object
 2   group_id        260000 non-null  int64 
 3   entity_name     260000 non-null  object
 4   entity_value    260000 non-null  object
 5   extracted_text  260000 non-null  object
dtypes: int64(2), object(4)
memory usage: 13.9+ MB


In [236]:
df.shape

(260000, 6)

In [237]:
df['entity_name'].unique()

array(['item_weight', 'item_volume', 'voltage', 'wattage',
       'maximum_weight_recommendation', 'height', 'depth', 'width'],
      dtype=object)

In [238]:
df.sample(4)


,Unnamed: 0,image_link,group_id,entity_name,entity_value,extracted_text
32469,2469,https://m.media-amazon.com/images/I/61-V0z5dnz...,681445,depth,4.8 inch,{'<OCR>': 'Roger!Profitede chaque instant.méri...
3702,3702,https://m.media-amazon.com/images/I/71kAteDLD-...,630869,item_weight,10.7 gram,{'<OCR>': 'Weight: 10.7g20cm 7.87inch2.2cm/0.8...
19899,4899,https://m.media-amazon.com/images/I/51nyPvtunH...,180726,height,10.0 millimetre,{'<OCR>': 'BEADNOVAWhat You Can Get?x 100Total...
27493,7493,https://m.media-amazon.com/images/I/61CYYPSSax...,944936,width,10.0 centimetre,{'<OCR>': 'Thankyou15 cm10 cm'}


In [239]:
df = df.drop(columns = ['Unnamed: 0'])

In [240]:
df.duplicated().sum()

7000

In [241]:
df = df.drop_duplicates()

In [242]:
df = df.drop(columns = ['image_link'])

In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253000 entries, 0 to 69999
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   group_id        253000 non-null  int64 
 1   entity_name     253000 non-null  object
 2   entity_value    253000 non-null  object
 3   extracted_text  253000 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.7+ MB


In [244]:
df['extracted_text'].iloc[2770]

"{'<OCR>': 'Portability500GWeight3MMEasy to carryThicknessFoldable into a thin sheet for easy portability'}"

In [245]:
def extract_values(text):
    try:
        data_dict = ast.literal_eval(text)
        return ' '.join(data_dict.values())
    except (ValueError, SyntaxError):
        return None  


df['extracted_values'] = df['extracted_text'].apply(extract_values)

In [246]:
df['extracted_values'].iloc[1790]

'UNIT FARMOTHERSUF 2000BLURPLE LEDDriver: LED BallastDriver: Non-isolated PowerLED Type: Unknown BrandLED Type; OSRAMLED Type : LIMitation of Lumens LedLifespan: 50,000HLiferspan: 100,000HLifers span: 20,000HD(Laggard TECH cause short lifespan)VOLTAGEVOLTAGESOLTAGE110V-277V110V -277V100-240VWATTS100WATSTS100PFPFPFPF180PPF114CRI220CRI70 CRICRICRIThe higher the easier to10 CRIFound the true color of your plants90CRIfound the color of the plantstoouq uue uue cojol ol. Aonl bijou2aOcBIОСКТИО ЛЛЕ цЫЕ СОJЕ, ч6БЕЛ, ПЛ6Е2Е1'

In [247]:
df.sample(4)


,group_id,entity_name,entity_value,extracted_text,extracted_values
29155,179080,voltage,36.0 volt,"{'<OCR>': 'tretwerkGreenway Akku417,6 Wh/36 V4...","tretwerkGreenway Akku417,6 Wh/36 V4,5 Std. Lad..."
4642,658892,width,3.0 centimetre,{'<OCR>': '3CM14CM'},3CM14CM
10671,297246,width,2.1 centimetre,"{'<OCR>': 'Package IncludesUSB 2.0USB 2,06.5cm...","Package IncludesUSB 2.0USB 2,06.5cm(2.5in)USB ..."
9836,630869,item_weight,8.5 gram,{'<OCR>': 'Circumference:25.3cm (Adjustable 6-...,Circumference:25.3cm (Adjustable 6-8.5 inch)10...


In [248]:
df = df.dropna()


In [249]:
df = df.drop(columns = ['extracted_text'])

In [250]:
df.head(4)

,group_id,entity_name,entity_value,extracted_values
0,748919,item_weight,500.0 gram,PROPÉS'NATUREINGREDIENT MÉAGERMULTI-USAGETERRE...
1,916768,item_volume,1.0 cup,TEARRIFIC!LEBENSMITTLECHT1Cupa DAYGEPRÄGTESDES...
2,459516,item_weight,0.709 gram,COMPOSITIONServing Size: (0.709 g) | Each serv...
3,459516,item_weight,0.709 gram,DUMPSONX-ABY1.0%1.00%2.00Vitamin EVitamin C (V...


In [251]:
df.shape

(252272, 4)

In [252]:
df['extracted_values'].iloc[0]

"PROPÉS'NATUREINGREDIENT MÉAGERMULTI-USAGETERRE DESOMMIÈRES100% NATURLAngue 100% paré à naturelle, l'erre deau parfumément pour un projetifrice de l'eau. La naturel de la naturel, des légumesparfumeur, blanc et sautifrice moquettesapples, gras et sauteur moquentesdeodoriser le linge.télégéant, bertoneeOngage confortable, Séden l'égé.Pour les couleurs de léger, de la couleur de la création de la bouleurPour l'amour de la vie en fronction de la fronciet100%Lait PRODUCE NOS2 500 gLait Progérationnelle de la méditeur d'un mégèneur.2 017070 78741"

In [253]:
df['extracted_values'].iloc[288]

'Adjustable Chain:55cm+5cm/22+2 inchesWidth: 3mmHigh Quality Black PlatedOFAMERICA37mmUNITEDGOD WETRUSTLIBERTYQUARTER DOLLAR30mm24mmNet weight:30g'

In [254]:
df['entity_name'].value_counts()

entity_name
item_weight                      101119
depth                             44693
width                             39506
height                            39234
voltage                            9319
wattage                            7642
item_volume                        7564
maximum_weight_recommendation      3195
Name: count, dtype: int64

In [255]:
import pandas as pd
import re

def clean_text(text):
    if isinstance(text, str):
       
        return re.sub(r'[^a-zA-Z0-9\s.,!?;:()\[\]\'"@#$%&*+-/=_<>|]', '', text)
    return text 
    
df['cleaned_extracted_values'] = df['extracted_values'].apply(clean_text)



In [256]:
df['cleaned_extracted_values'] = df['cleaned_extracted_values'].str.lower()

In [257]:
df = df.drop(columns = ['extracted_values'])

In [258]:
df.sample(4)

,group_id,entity_name,entity_value,cleaned_extracted_values
67387,347404,height,9.25 inch,rope length: 78.74 inch2.56 inch9.25 inch
51301,752266,wattage,30.0 watt,cautionledfor more interesting functions ofthe...
21461,347404,width,19.29 inch,12.2in / 31 cm19.29in / 49 cm
28046,593646,width,90.0 centimetre,matriaupolytrphtalated'thylnecouleurblancmatri...


In [259]:
df['cleaned_extracted_values'].iloc[222]

'ccorccorpeinado de plantasgastricoort pakistantecoorte negrocon nariana75gpeso neto100 g100 gcoor100 gt verdecon jazzinrooibos100 g'

In [260]:
df.isna().sum()

group_id                    0
entity_name                 0
entity_value                0
cleaned_extracted_values    0
dtype: int64

In [261]:
import re

def add_space_before_numbers(text):

    updated_text = re.sub(r'(?<![0-9\s.])(\d)', r' \1', text)
    return updated_text

In [262]:
df['cleaned_extracted_values'] = df['cleaned_extracted_values'].apply(add_space_before_numbers)

In [263]:
df['cleaned_extracted_values'].iloc[222]

'ccorccorpeinado de plantasgastricoort pakistantecoorte negrocon nariana 75gpeso neto 100 g 100 gcoor 100 gt verdecon jazzinrooibos 100 g'

In [264]:
df.iloc[980]

group_id                                                               308856
entity_name                                                       item_weight
entity_value                                                      14 kilogram
cleaned_extracted_values    made withsellectednaturalingredientsleminepuri...
Name: 981, dtype: object

In [265]:
df = df[~df['entity_value'].str.contains('horsepower', case=False, na=False)]

In [266]:
df['cleaned_extracted_values'].iloc[980]

'pumpednitric onstim free& pump.performance & pumpperformafreakathletics 320g strawberry 20- 40food supplementslushservings'

In [267]:
df.isna().sum()

group_id                    0
entity_name                 0
entity_value                0
cleaned_extracted_values    0
dtype: int64

In [268]:
df.shape

(252031, 4)

In [269]:
df.iloc[11327]

group_id                                                               501250
entity_name                                                       item_weight
entity_value                                                     2.2 kilogram
cleaned_extracted_values    do not coverdo not powerdo not coverdonter pow...
Name: 11384, dtype: object

In [270]:
df = df[~df['entity_value'].str.contains(r'\[.*\]', regex=True)]

In [271]:
df.shape

(248804, 4)

In [272]:
df['cleaned_extracted_values'].sample(8)

54793    parameter detailsnot easy to lose hair 15.75in...
66800     8mmproduct parameters 0.31[in]r 7s halogen bu...
21638    free and roomy spaceprovides a good space insi...
27813         18"x 24" inches 24" inchessalton 18" incches
265                4- 1/ 2" 1- 3/ 4" 1 - 3/ 4"- 1 - 3 / 4"
50584    curtainitem number: 134517vida y.comfrpensez o...
11439    product dimensions 2 gang standard size 4.57" ...
50740    pedal size : 2324cm 110cm 23cm 125cm 71cm 90cm...
Name: cleaned_extracted_values, dtype: object

In [273]:
import pandas as pd
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    if isinstance(text, str): 
        return " ".join(word for word in text.split() if word.lower() not in stop_words)
    return text  




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [274]:
df['cleaned_extracted_values'] = df['cleaned_extracted_values'].apply(remove_stopwords)

In [275]:
df.shape

(248804, 4)

In [276]:
df = df.sample(frac=1).reset_index(drop=True)

In [277]:
df.head()

,group_id,entity_name,entity_value,cleaned_extracted_values
0,180726,height,25.0 millimetre,"d: 25mm( 1"")ih: 21mm/ 0.84""oh: 23mm 8: 2.6mm/o..."
1,752266,voltage,1500.0 volt,extrastorceextrostor*extrasto*extrastarlay clu...
2,731432,item_weight,30 gram,tune bodypure himalayantune bodypure himalayan...
3,149159,height,8.65 inch,8.65 / 22 cm 8. 65 / 22 cm
4,752266,voltage,3.2 volt,product detailthicker heatsink.safeand waterpr...


In [278]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [279]:

def common_mistake(unit):
    if unit in allowed_units:
        return unit
    if unit.replace('ter', 'tre') in allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in allowed_units:
        return unit.replace('feet', 'foot')
    return unit

In [280]:
df['entity_value'] = df['entity_value'].apply(common_mistake)

In [281]:
df.shape


(248804, 4)

In [282]:
df['cleaned_extracted_values'].iloc[980]

'5.5 ( 14 cm)'

In [283]:
df[df['entity_name'] == 'width']['cleaned_extracted_values'].iloc[80]

'13.5cm/ 5.31in 7cm/ 2.76in'

In [284]:
import re

def correct_weight_separation(input_text):

    units = [
        "g", "kg", "mg", "ml", "l", "oz", "lb", "pcs", "ct", "cfqty",  
        "gram", "kilogram", "microgram", "milligram", "litre", "liter", "ounce", "pound", "ton", "piece", "pieces" 
    ]
    

    unit_pattern = "|".join(units) 
    pattern = rf"(\d+(\.\d+)?(?:{unit_pattern}))"  
    
    corrected_text = re.sub(pattern, r" \1 ", input_text)

    corrected_text = re.sub(r"\s{2,}", " ", corrected_text).strip()
    
    return corrected_text



In [285]:
df['cleaned_extracted_values'] = df['cleaned_extracted_values'].apply(correct_weight_separation)

In [286]:

def correct_dimension_separation(input_text):

    units = [
        "cm", "mm", "m", "in", "inch", "ft", "yard", 
        "centimetre", "millimetre", "metre", "foot", "yard", "inches"  
    ]   
    unit_pattern = "|".join(units) 
    pattern = rf"(\d+(\.\d+)?(?:{unit_pattern}))"

    corrected_text = re.sub(pattern, r" \1 ", input_text)

    corrected_text = re.sub(r"\s{2,}", " ", corrected_text).strip()
    
    return corrected_text


In [287]:
df['cleaned_extracted_values'] = df['cleaned_extracted_values'].apply(correct_dimension_separation)

In [288]:
def correct_quantity_separation(input_text):

    voltage_units = [
        "kV", "mV", "V", "kilovolt", "millivolt", "volt" , "v" , "kv" , "mv"
    ]
    
    wattage_units = [
        "kW", "W", "kilowatt", "watt" , "w" , "kw"
    ]
    
    volume_units = [
        "cl", "centilitre", "ft³", "cubic foot", "in³", "cubic inch", "cup", "dl", "decilitre",
        "fl oz", "fluid ounce", "gal", "gallon", "imp gal", "imperial gallon", "l", "litre", "litres"
        "µl", "microlitre", "ml", "millilitre", "pt", "pint", "qt", "quart"
    ]
    
    voltage_pattern = r"(\d+(\.\d+)?(?:{}))".format("|".join(voltage_units))
    wattage_pattern = r"(\d+(\.\d+)?(?:{}))".format("|".join(wattage_units))
    volume_pattern = r"(\d+(\.\d+)?(?:{}))".format("|".join(volume_units))

    def add_space_after_unit(pattern, text):
        return re.sub(pattern, r" \1 ", text)
    
    corrected_text = input_text
    corrected_text = add_space_after_unit(voltage_pattern, corrected_text)
    corrected_text = add_space_after_unit(wattage_pattern, corrected_text)
    corrected_text = add_space_after_unit(volume_pattern, corrected_text)
    
    corrected_text = re.sub(r"\s{2,}", " ", corrected_text).strip()
    
    return corrected_text


In [289]:
df['cleaned_extracted_values'] = df['cleaned_extracted_values'].apply(correct_quantity_separation)

In [291]:
df['cleaned_extracted_values'].iloc[301]

'premiumfightwear.compremiuma.compremummade u.s.a.p.m.premiumfightwalking.com.commadein loving co.l.f.c.lifestremiumengage independenthenthing co ., lifesthenthe.com ., made england ., s.a ., e.c ., u.p ., n.e ., w.e.8.1.6 quince ., b.i.g.'

In [293]:

df['Contains_Numerical'] = df['cleaned_extracted_values'].str.contains(r'\d')

df['Contains_Numerical'].value_counts()

Contains_Numerical
True     243477
False      5327
Name: count, dtype: int64

In [301]:
df = df[df['Contains_Numerical'] == True]

In [302]:
df = df.drop(columns = ['Contains_Numerical'])

In [304]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243477 entries, 0 to 248803
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   group_id                  243477 non-null  int64 
 1   entity_name               243477 non-null  object
 2   entity_value              243477 non-null  object
 3   cleaned_extracted_values  243477 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.3+ MB


In [311]:
train = df[0:215000]
test = df[215000:]

In [312]:
train.to_csv("Final_train_data.csv")

In [313]:
test.to_csv("Final_test_data.csv")

In [314]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28477 entries, 219661 to 248803
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   group_id                  28477 non-null  int64 
 1   entity_name               28477 non-null  object
 2   entity_value              28477 non-null  object
 3   cleaned_extracted_values  28477 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [316]:
test['entity_name'].unique()

array(['item_weight', 'width', 'height', 'depth', 'voltage',
       'item_volume', 'maximum_weight_recommendation', 'wattage'],
      dtype=object)

In [317]:
d = pd.read_csv("train.csv")

In [318]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB
